In [1]:
import pandas as pd

In [104]:
import h2o
from h2o.automl import H2OAutoML

In [105]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 30 mins
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_angus_vpj8go
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.682 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [106]:
cols= ['buying','maint','doors','persons','lug_boot','safety','class_value']


In [107]:
df = pd.read_csv('.//data/car.data', names=cols)

In [108]:
df.describe()

,buying,maint,doors,persons,lug_boot,safety,class_value
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,432,576,576,576,1210


In [109]:
for col in cols:
    print(df[col].value_counts())  

vhigh    432
high     432
med      432
low      432
Name: buying, dtype: int64
vhigh    432
high     432
med      432
low      432
Name: maint, dtype: int64
2        432
3        432
4        432
5more    432
Name: doors, dtype: int64
2       576
4       576
more    576
Name: persons, dtype: int64
small    576
med      576
big      576
Name: lug_boot, dtype: int64
low     576
med     576
high    576
Name: safety, dtype: int64
unacc    1210
acc       384
good       69
vgood      65
Name: class_value, dtype: int64


In [110]:
df.drop(['persons'], axis=1, inplace=True) # this column is not used in prediction as per the instructions

In [111]:
df.head()

,buying,maint,doors,lug_boot,safety,class_value
0,vhigh,vhigh,2,small,low,unacc
1,vhigh,vhigh,2,small,med,unacc
2,vhigh,vhigh,2,small,high,unacc
3,vhigh,vhigh,2,med,low,unacc
4,vhigh,vhigh,2,med,med,unacc


In [112]:
# convert doors column to pure string so that it will not be interpret as numeric by h2o

In [113]:
df['doors'] = [str(x) + 'doors' for x in df['doors']]

Pass data to H2O for training

In [114]:
hf = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [115]:
hf.describe()

Rows:1728
Cols:6

,buying,maint,doors,lug_boot,safety,class_value
type,enum,enum,enum,enum,enum,enum
mins,,,,,,
mean,,,,,,
maxs,,,,,,
sigma,,,,,,
zeros,,,,,,
missing,0,0,0,0,0,0
0,vhigh,vhigh,2doors,small,low,unacc
1,vhigh,vhigh,2doors,small,med,unacc
2,vhigh,vhigh,2doors,small,high,unacc


In [116]:
train, test = hf.split_frame(ratios=[0.8], seed = 88)

In [117]:
aml = H2OAutoML(max_models =25, seed = 88, balance_classes=True)

In [118]:
aml.train(training_frame = hf, y = 'buying')

AutoML progress: |██
23:49:22.577: GLM_1_AutoML_6_20221120_234916 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException

█████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_6_20221120_234916

No summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.5988169550202618
RMSE: 0.7738326401879555
LogLoss: 2.4514502121781523
Null degrees of freedom: 1727
Residual degrees of freedom: 1450
Null deviance: 4791.033312030129
Residual deviance: 8472.21193328783
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
high    low    med    vhigh    Error     Rate
------  -----  -----  -------  --------  -------------
338     0      37     57       0.217593  94 / 432
301     23     75     33       0.946759  409 / 432
320     0      75     37       0.826389  357 / 432
316     0      37     79       0.81713   353 / 432
1275    23     224    206      0.701968  1,213 / 1,728

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.298032
2    0.569444
3    0.827546
4    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.3298008378780402
RMSE: 0.5742828901143061
LogLoss: 0.8976502077675527
Null degrees of freedom: 1727
Residual degrees of freedom: 1474
Null deviance: 4793.673673391488
Residual deviance: 3102.279118044662
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
high    low    med    vhigh    Error     Rate
------  -----  -----  -------  --------  -----------
288     77     34     33       0.333333  144 / 432
111     228    45     48       0.472222  204 / 432
62      31     223    116      0.483796  209 / 432
45      42     98     247      0.428241  185 / 432
506     378    400    444      0.429398  742 / 1,728

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.570602
2    0.875
3    0.98206
4    1

Cross-Validation Metrics Summary: 
                         mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.570287  0.0244529  0.575581      0.591716      0.552778      0.593567      0.537791
auc                      nan       0          nan           nan           nan           nan           nan
err                      0.429713  0.0244529  0.424419      0.408284      0.447222      0.406433      0.462209
err_count                148.6     10.8766    146           138           161           139           159
logloss                  0.895693  0.0403987  0.893363      0.867796      0.883473      0.868467      0.965363
max_per_class_error      0.507936  0.0226786  0.488095      0.522222      0.48913       0.54023       0.5
mean_per_class_accuracy  0.570337  0.0245088  0.574242      0.596963      0.553033      0.589137      0.538311
mean_per_class_error     0.429663  0.0245088  0.425758      0.403037      0.446967      0.410863      0.461689
mse                      0.329284  0.016552   0.32716       0.319618      0.328334      0.314255      0.357054
null_deviance            958.735   22.9916    953.939       938.04        998.185       949.584       953.926
pr_auc                   nan       0          nan           nan           nan           nan           nan
r2                       0.73591   0.0161418  0.73944    

In [119]:
aml.leaderboard.head()

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_1_AutoML_6_20221120_234916,0.429398,0.89765,0.574283,0.329801
StackedEnsemble_BestOfFamily_1_AutoML_6_20221120_234916,0.541088,1.15177,0.661415,0.43747
GBM_grid_1_AutoML_6_20221120_234916_model_1,0.657407,1.32447,0.731654,0.535318
GBM_grid_1_AutoML_6_20221120_234916_model_2,0.663194,1.30245,0.723647,0.523665
GBM_1_AutoML_6_20221120_234916,0.666667,1.27866,0.716931,0.513991
XGBoost_grid_1_AutoML_6_20221120_234916_model_3,0.666667,1.289,0.719326,0.517429
XGBoost_grid_1_AutoML_6_20221120_234916_model_5,0.69213,1.24675,0.705484,0.497707
GBM_grid_1_AutoML_6_20221120_234916_model_3,0.71875,1.2651,0.711813,0.506677
DeepLearning_grid_1_AutoML_6_20221120_234916_model_1,0.71875,2.0652,0.76371,0.583252
XGBoost_1_AutoML_6_20221120_234916,0.722222,1.27436,0.71185,0.506731


In [120]:
best_model = aml.get_best_model()

In [121]:
best_model.model_performance(test)

ModelMetricsMultinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.5957687967635649
RMSE: 0.7718606070810745
LogLoss: 2.461384896039459
Null degrees of freedom: 348
Residual degrees of freedom: 71
Null deviance: 967.6334640616857
Residual deviance: 1718.0466574354803
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
high    low    med    vhigh    Error     Rate
------  -----  -----  -------  --------  ---------
69      0      6      9        0.178571  15 / 84
59      3      18     6        0.965116  83 / 86
55      0      15     9        0.810127  64 / 79
74      0      8      18       0.82      82 / 100
257     3      47     42       0.69914   244 / 349

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.30086
2    0.590258
3    0.830946
4    1

Use the best model to make a prediction

In [122]:
df_predict=pd.DataFrame([['high', '4doors', 'big', 'high', 'good'],],columns=['maint', 'doors', 'lug_boot', 'safety', 'class_value'])

In [123]:
df_predict

,maint,doors,lug_boot,safety,class_value
0,high,4doors,big,high,good


In [124]:
hf_predict = h2o.H2OFrame(df_predict)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [125]:
best_model.predict(hf_predict)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,high,low,med,vhigh
low,0.331549,0.668355,9.67754e-05,4.98995e-08
